In [2]:
from __future__ import absolute_import, division
from sklearn.decomposition import TruncatedSVD

import Model
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
import senteval
import pickle
import copy
import operator

PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

maxLength = 30    
wordDim = 300    # 300 
num_neurons = [2*wordDim,2*wordDim]   # [600,600f]
extractLayer = ['X_input_second','X_input_mask_2']
kernel_reg = 0.001
batch_size = 400
merge_mode = 'concat'
checkpoint =  "../Checkpoint_summary/QT-Glove-Epoch-1-loss-0.6732-val_loss-0.7173.hdf5"  # Change 
embeddingPath = "../Embedding/glove.840B.300d.txt"  # Change 

processedPath = "../processed/"
tokenizerFile = 'tokenizer.txt'
embeddingFile =  'embedding_matrix.txt'
embeddingSize = 678318
dicFile = 'wordDic.txt'
marked_token = "maskedtoken"
n_component = 3


model = []

with open(processedPath + tokenizerFile , "rb") as input_file:
    t = pickle.load(input_file)

with open(processedPath + embeddingFile , "rb") as input_file:
    embedding_matrix = pickle.load(input_file) 
    
vocab_size = len(embedding_matrix)
 
count = 0
print("Load embedding")
embeddings_index = dict()
f = open(embeddingPath  , 'r', errors = 'ignore', encoding='utf8')  # "word2vec.6B.50d.txt" for test 
for line in f:

    splitLine = line.split(' ')  # For 3B  splitLine = line.split()
    word = splitLine[0]
    coefs = np.array([float(val) for val in splitLine[1:]])
    embeddings_index[word] = coefs
f.close()
print("Finish loading")

# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    global embeddings_index
    global count
    global t
    global embedding_matrix
    global model
    
    sentences = [' '.join(s) for s in batch]
            
    sentences  = Model.preprocessing(sentences)
        
    sentenceList = []
    for sentence in sentences:
        print(sentence)
        wordList = []
        for vocab in sentence:
            try:
                wordList.append(embeddings_index[vocab])
            except:
                wordList.append(np.zeros(wordDim))
            
        while(len(wordList)< n_component):
            wordList.append(np.zeros(wordDim))
        
        svd = TruncatedSVD(n_components=n_component, n_iter=5, random_state=42)
        
        svd.fit(wordList)
        for i in range(n_component):
            svd.components_[i] = svd.components_[i]*svd.singular_values_[i]
        output =  svd.components_.flatten()
        #output = output.tolist() + svd.singular_values_.tolist()
        sentenceList.append(output)

    sentences = t.texts_to_sequences(sentences)
    sentences = pad_sequences(sentences, maxlen=maxLength, padding='post')
    # Restore model 
    
    if count == 0:
        print("build model")
        model = Model.buildModel(sentences,kernel_reg,num_neurons,merge_mode, embeddingSize,maxLength, wordDim,np.zeros([embeddingSize,wordDim]),False)
        model.load_weights(checkpoint)
        count += 1

    # Extract th output 
    output = Model.extractHiddenState(extractLayer,model,sentences)
    output = np.concatenate((output[0],output[1]),axis=1)
    output = np.concatenate((output,sentenceList),axis=1)
    print(len(output[0]))
    return output


Using TensorFlow backend.


Load embedding
Finish loading


In [9]:
checkpoint =  "../Checkpoint_summary/QT-Glove-Epoch-1-loss-0.6732-val_loss-0.7173.hdf5"

In [ ]:
tasks = ['MR','MPQA']
result = []
for task in tasks:
    print(task)
    params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
    params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 128,
                                     'tenacity': 5, 'epoch_size': 5}
    # Set up logger
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks =    [task]  # ['CR','MPQA','TREC','MRPC']
    # 'MR', 'CR' ,'MPQA', 'SUBJ','TREC', 'MRPC','SICKEntailment', 'SICKRelatedness'  ,'TREC','MRPC'
    results = se.eval(transfer_tasks)
    result.append(results)


MR
Pre processing Data
['obvious']
['horrible']
['crummy']
['execrable', '.']
['shallow', '.']
['eh', '.']
['abysmally', 'pathetic']
['terrible', '.']
['wishy', 'washy', '.']
['spiderman', 'rocks']
['delightfully', 'rendered']
['surprisingly', 'insightful']
['touchm', 'i', '!']
['refreshing', '.']
['fantastic', '!']
['calculated', 'swill', '.']
['thoroughly', 'awful', '.']
['truly', 'terrible', '.']
['by', 'the', 'numbers', 'yarn', '.']
['amazingly', 'dopey', '.']
['crikey', 'indeed', '.']
['so', 'so', 'entertainment', '.']
['cinematic', 'poo', '.']
['extremely', 'bad', '.']
['woefully', 'pretentious', '.']
['warmed', 'over', 'hash', '.']
['lacks', 'depth', '.']
['under', 'rehearsed', 'and', 'lifeless']
['unwieldy', 'contraption', '.']
['feeble', 'comedy', '.']
['disjointed', 'parody', '.']
['two', 'bit', 'potboiler', '.']
['painfully', 'padded', '.']
['dramatically', 'lackluster', '.']
['incoherence', 'reigns', '.']
['mildly', 'amusing', '.']
['fairly', 'run', 'of', 'the', 'mill', '.'

3300
Pre processing Data
['a', 'delightful', 'coming', 'of', 'age', 'story', '.']
['an', 'absurdist', 'spider', 'web', '.']
['familiar', 'but', 'utterly', 'delightful', '.']
['a', 'no', 'holds', 'barred', 'cinematic', 'treat', '.']
['poignant', 'and', 'delicately', 'complex', '.']
['a', 'captivating', 'new', 'film', '.']
['the', 'performances', 'are', 'remarkable', '.']
['melodrama', 'with', 'a', 'message', '.']
['sweet', 'and', 'memorable', 'film', '.']
['a', 'creaky', 'staircase', 'gothic', '.']
['surprisingly', 'powerful', 'and', 'universal', '.']
['gently', 'humorous', 'and', 'touching', '.']
['reassuring', ',', 'retro', 'uplifter', '.']
['lavishly', ',', 'exhilaratingly', 'tasteless', '.']
['slight', 'but', 'enjoyable', 'documentary', '.']
['pretty', 'good', 'little', 'movie', '.']
['a', 'stirring', 'road', 'movie', '.']
['adaptation', 'is', 'simply', 'brilliant', '.']
['a', 'smart', 'little', 'indie', '.']
['an', 'extraordinary', 'dramatic', 'experience', '.']
['strange', 'and', 

['works', 'as', 'pretty', 'contagious', 'fun', '.']
['a', 'lovably', 'old', 'school', 'hollywood', 'confection', '.']
['a', 'wry', ',', 'affectionate', 'delight', '.']
['an', 'enthralling', ',', 'entertaining', 'feature', '.']
['not', 'so', 'much', 'farcical', 'as', 'sour', '.']
['the', 'action', 'clichm', 'is', 'just', 'pile', 'up', '.']
['gooding', 'offers', 'a', 'desperately', 'ingratiating', 'performance', '.']
['the', 'worst', 'film', 'of', 'the', 'year', '.']
['glazed', 'with', 'a', 'tawdry', 'b', 'movie', 'scum', '.']
['the', 'pool', 'drowned', 'me', 'in', 'boredom', '.']
['wait', 'for', 'it', 'to', 'hit', 'cable', '.']
['bluto', 'blutarsky', ',', 'we', 'miss', 'you', '.']
['there', 'is', 'so', 'much', 'plodding', 'sensitivity', '.']
['feels', 'shrill', ',', 'simple', 'and', 'soapy', '.']
['a', 'direct', 'to', 'void', 'release', ',', 'heading', 'nowhere', '.']
['a', 'fifty', 'car', 'pileup', 'of', 'cliches', '.']
['a', 'rather', 'tired', 'exercise', 'in', 'nostalgia', '.']
['hip

3300
Pre processing Data
['a', 'witty', ',', 'low', 'key', 'romantic', 'comedy', '.']
['a', 'winning', 'and', 'wildly', 'fascinating', 'work', '.']
['what', 'i', 'saw', ',', 'i', 'enjoyed', '.']
['a', 'well', 'put', 'together', 'piece', 'of', 'urban', 'satire', '.']
['the', 'gags', 'are', 'often', 'a', 'stitch', '.']
['uneven', 'but', 'a', 'lot', 'of', 'fun', '.']
['a', 'tasty', 'slice', 'of', 'droll', 'whimsy', '.']
['a', 'very', 'charming', 'and', 'funny', 'movie', '.']
['spend', 'your', 'benjamins', 'on', 'a', 'matinee', '.']
['a', 'pleasant', 'piece', 'of', 'escapist', 'entertainment', '.']
['witty', ',', 'vibrant', ',', 'and', 'intelligent', '.']
['all', 'three', 'women', 'deliver', 'remarkable', 'performances', '.']
['a', 'gentle', 'and', 'engrossing', 'character', 'study', '.']
['ferrara', "'s", 'best', 'film', 'in', 'years', '.']
['romantic', ',', 'riveting', 'and', 'handsomely', 'animated', '.']
['hugely', 'accomplished', 'slice', 'of', 'hitchcockian', 'suspense', '.']
['the',

3300
Pre processing Data
['plays', 'like', 'the', 'old', 'disease', 'of', 'the', 'week', 'small', 'screen', 'melodramas', '.']
['an', 'awkwardly', 'contrived', 'exercise', 'in', 'magic', 'realism', '.']
['one', 'of', 'the', 'worst', 'films', 'of', '2002', '.']
['comes', 'across', 'as', 'a', 'fairly', 'weak', 'retooling', '.']
['earnest', 'and', 'heartfelt', 'but', 'undernourished', 'and', 'plodding', '.']
['more', 'a', 'gunfest', 'than', 'a', 'rock', 'concert', '.']
['a', 'characteristically', 'engorged', 'and', 'sloppy', 'coming', 'of', 'age', 'movie', '.']
['loud', ',', 'silly', ',', 'stupid', 'and', 'pointless', '.']
['a', 'loquacious', 'and', 'dreary', 'piece', 'of', 'business', '.']
['makes', '98', 'minutes', 'feel', 'like', 'three', 'hours', '.']
['low', 'comedy', 'does', "n't", 'come', 'much', 'lower', '.']
['it', "'s", 'just', 'rather', 'leaden', 'and', 'dull', '.']
['hollywood', "'s", 'answer', 'to', 'an', 'air', 'ball', '.']
['a', 'porn', 'film', 'without', 'the', 'sex', 'sce

...

3300
Pre processing Data
['now', 'more', 'people', 'will', 'be', 'able', 'to', 'judge']
['is', 'realized', 'and', 'the', 'cause', 'of', 'justice', 'wins']
['stand', 'with', 'them', 'in', 'the', 'same', 'trench', 'steadfastly']
['agreed', 'the', 'adoption', 'of', 'a', 'decision', 'for', 'approval']
['bringing', 'an', 'end', 'to', 'terrorism', 'and', 'the', 'taliban']
['serious', ',', 'calm', 'and', 'forward', 'looking', 'approach']
['end', 'of', 'the', 'era', 'of', 'violence', 'and', 'counterviolence']
['that', 'there', 'is', 'a', 'future', 'to', 'be', 'built']
['skillfully', 'adapted', 'themselves', 'to', 'the', 'most', 'difficult', 'circumstances']
['had', 'the', 'advantage', 'for', 'washington', 'of', 'weakening', 'opec']
['can', 'not', 'imagine', 'anything', 'as', 'beautiful', 'as', 'that']
['the', 'most', 'reasonable', 'path', 'for', 'the', 'time', 'being']
['at', 'least', 'they', "'re", 'coming', 'out', 'with', 'something']
['to', 'ensure', 'the', 'full', 'exercise', 'of', 'democr

["o'keefe", 'jumps', 'with', 'both', 'feet', 'into', 'this', 'fiscal', 'morass']
['appear', 'to', 'make', 'it', 'easier', 'to', 'rig', 'the', 'vote']
['uses', 'a', 'double', 'standard', 'in', 'its', 'definition', 'of', 'terrorism']
['world', "'s", 'most', 'prominent', 'self', 'styled', 'palestinian', 'refugee']
['what', 'begins', 'to', 'happen', 'when', 'we', 'hate', 'our', 'friends']
['must', 'push', 'for', 'punitive', 'action', 'against', 'mr', '.', 'mugabe']
['does', 'not', 'do', 'what', 'it', 'is', 'supposed', 'to', 'do']
['in', 'a', 'way', 'that', 'can', 'not', 'be', 'taken', 'seriously']
['unprecedented', 'in', 'the', 'history', 'of', 'the', 'comity', 'of', 'nations']
['tries', 'to', 'play', 'the', 'role', 'of', 'a', 'world', 'policeman']
['the', 'like', 'of', 'which', 'has', 'never', 'been', 'seen', 'before']
['not', 'a', 'nation', 'that', 'is', 'able', 'to', 'abide', 'by']
['look', 'like', 'a', 'chicken', 'farm', 'run', 'by', 'a', 'sadist']
['it', 'had', 'to', 'happen', 'and', 

['did', 'away', 'with', 'the', 'discrimination', 'that', 'had', 'existed', 'for', 'years']
['an', 'advocate', 'of', 'nations', "'", 'right', 'to', 'choose', 'their', 'leaders']
['his', 'victory', 'was', 'a', 'triumph', 'against', 'the', 'west', 'and', 'whites']
['with', 'good', 'economic', 'management', ',', 'israel', 'should', 'have', 'little', 'trouble']
['is', 'giving', 'the', 'civil', 'rights', 'advocates', 'a', 'hearty', 'thumbs', 'up']
['the', 'brutality', 'with', 'which', 'this', 'closure', 'was', 'implemented', 'was', 'unheard', 'of']
['sat', 'back', 'with', 'arms', 'folded', 'would', 'have', 'made', 'us', 'all', 'hostages']
['affecting', 'their', 'medical', ',', 'economic', ',', 'educational', ',', 'and', 'social', 'life']
['is', 'not', ',', 'in', 'the', 'first', 'place', ',', 'willing', 'to', 'recognize']
['greenpeace', 'is', 'here', 'to', 'voice', 'the', 'concerns', 'of', 'millions', 'of', 'people']
['higher', 'than', 'any', 'time', 'in', 'the', 'past', 'several', 'hundred',

['would', 'cause', 'japan', 'and', 'the', 'republic', 'of', 'korea', '(', 'rok', ')', 'feel', 'disturbed']
['increase', 'the', 'palestinians', "'", 'suffering', 'and', 'encourage', 'the', 'israeli', 'right', 'wing', 'stands']
['her', 'former', 'friend', 'as', 'an', 'enemy', 'and', 'her', 'former', 'enemy', 'as', 'a', 'friend']
['notwithstanding', 'the', 'labyrinth', 'of', 'constitutional', 'provisions', 'loaded', 'against', 'a', 'free', 'and', 'fair', 'election']
['taken', 'the', 'israeli', 'government', 'and', 'consequently', 'both', 'the', 'israeli', 'and', 'palestinian', 'peoples', 'hostage']
['not', 'just', 'simply', 'to', 'determine', 'if', 'there', 'were', 'a', 'coup', "d'etat", 'or', 'not']
['those', 'who', 'seek', 'to', 'attack', 'and', 'destroy', 'law', 'and', 'order', 'and', 'legitimate', 'government']
['the', 'failed', 'coup', 'attempts', 'that', 'the', 'united', 'states', 'first', 'blessed', 'and', 'then', 'disavowed']
['his', 'ship', 'has', 'been', 'sailing', 'towards', 'a

['it', 'did', 'not', 'apply', 'even', 'to', 'black', 'people', 'held', 'in', 'jail', 'in', 'the', 'southern', 'united', 'states', 'until', 'a', 'few', 'years', 'ago']
['are', 'not', 'guessing', 'correctly', 'about', 'the', 'dilemma', 'of', 'the', 'chicken', 'or', 'the', 'egg', 'into', 'which', 'the', 'argentine', 'crisis', 'has', 'become', 'converted']
['appears', 'to', 'be', 'another', 'faith', 'based', 'initiative', ':', 'we', 'should', 'have', 'faith', 'that', 'major', 'corporations', 'will', 'line', 'up', 'to', 'volunteer']
['though', 'mr', '.', 'tobin', 'has', 'finally', 'mastered', 'the', 'diapering', 'process', ',', 'his', 'baby', 'who', 'is', '15', 'years', 'old', 'is', 'not', 'amused']
['if', 'one', 'goes', 'by', 'the', 'us', 'logic', ',', 'only', 'the', 'us', 'spy', 'plane', 'is', 'permitted', 'to', 'spy', 'at', 'other', 'people', "'s", 'doorsteps']
['and', 'yet', 'it', 'was', 'unable', ',', 'despite', 'all', 'its', 'forces', ',', 'weapons', ',', 'and', 'measures', ',', 'to',

In [11]:
result

[{'CR': {'devacc': 81.63, 'acc': 80.11, 'ndev': 3775, 'ntest': 3775}},
 {'MRPC': {'devacc': 74.48,
   'acc': 72.41,
   'f1': 80.12,
   'ndev': 4076,
   'ntest': 1725}},
 {'SUBJ': {'devacc': 91.86, 'acc': 91.46, 'ndev': 10000, 'ntest': 10000}},
 {'TREC': {'devacc': 81.22, 'acc': 86.4, 'ndev': 5452, 'ntest': 500}}]

In [ ]:
embeddings_index = dict()

if len(embeddings_index) == 0:
    f = open(embeddingPath  , 'r', errors = 'ignore', encoding='utf8')  # "word2vec.6B.50d.txt" for test 
    for line in f:

        splitLine = line.split(' ')  # For 3B  splitLine = line.split()
        word = splitLine[0]
        coefs = np.array([float(val) for val in splitLine[1:]])

    #     values = line.split()
    #     word = values[0]
    #     coefs = asarray(values[1:], dtype='float32' )
        embeddings_index[word] = coefs
    f.close()

In [ ]:
try:
    embeddings_index['third']
except:
    print("what")

In [ ]:
wordSentence = ['this','is','a','test','how']
wordList = []
for vocab in wordSentence:
    wordList.append(embeddings_index[vocab])


In [ ]:
wordList

In [ ]:
svd = TruncatedSVD(n_components=4, n_iter=5, random_state=42)
svd.fit(wordList)
output =  svd.components_.flatten()

In [ ]:
svd.components_.shape

In [ ]:
x = [2,3]
y = [1]

x + y